*   Nama : Aurellia Zhullvita Amanullah
*   NIM  : 220411100200
*   Kelas: PPW A



# **Vector Space Model (VSM)**

Vector Space Model (VSM) adalah model yang digunakan untuk menggambarkan dokumen teks sebagai kumpulan angka atau vektor di dalam ruang. Dalam model ini, setiap kata dalam dokumen dianggap sebagai sebuah dimensi di ruang tersebut.
Dalam VSM, semakin banyak kata yang sama muncul di dua dokumen, maka jarak antar titik yang mewakili dua dokumen tersebut semakin dekat. Jadi, VSM ini digunakan untuk mengukur seberapa mirip dua dokumen atau seberapa relevan sebuah dokumen dengan pertanyaan yang kita ajukan (query).Biasanya, frekuensi kata digunakan untuk menentukan nilai angka dalam vektor, dan cosine similarity sering dipakai untuk menghitung kemiripan antara dua dokumen.

**Proses Pembentukan data ke bentuk VSM (Vector Space Model)**

In [ ]:
!pip install -q Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
# library awal untuk perhitungan dan pengolahan teks
import numpy as np
import re
import pandas as pd

# monitoring
from tqdm import tqdm

# library untuk praproses teks
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# library untuk proses pembentukan vsm
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Dikarenakan pada praproses teks akan melakukan stemming, maka perlu dilakukan install library Sastrawi terlebih dahulu untuk melakukan stemming bahasa indonesia.

numpy digunakan untuk pengolahan angka dan perhitungan matematik

re (regular expression) digunakan untuk mengenali pola kata/kalimat

nltk (natural language toolkit) berfungsi untuk melakukan proses yang berkaitan dengan bahasa (teks).

sklearn digunakan dalam pemrosesan data untuk kebutuhan machine learning atau data science. Dalam tugas ini, menggunakan TfidfTransformer dan CountVectorizer untuk menghitung Term Frequency dan Invers Document Frequency, serta Pipeline untuk mengelola data list lebih optimal

**Load Data**

In [ ]:
main_df = pd.read_csv('https://raw.githubusercontent.com/meinhere/ppw/master/publish/tugas-2/data_berita.csv', delimiter=',')
main_df

,No,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,Simak Jadwal dan Lokasi SIM Keliling di Jakart...,"JAKARTA, KOMPAS.com - Surat Izin Mengemudi (S...",07/09/2024,OTOMOTIF
1,2,[POPULER OTOMOTIF] Diskon Motor Honda Septembe...,"JAKARTA, KOMPAS.com - Banyak pembaca yang ingi...",07/09/2024,OTOMOTIF
2,3,"Cek Saldo Minimal BRI, BNI, BCA, Mandiri, dan BSI","JAKARTA, KOMPAS.com - Penting bagi calon nasab...",06/09/2024,MONEY
3,4,"KAI Uji Coba Teknologi ""Face Recognition Board...",KOMPAS.com - PT Kereta Api Indonesia (KAI) Div...,06/09/2024,MONEY
4,5,OJK Blokir 10.890 Entitas Keuangan Ilegal Seja...,"JAKARTA, KOMPAS.com - Otoritas Jasa Keuangan (...",06/09/2024,MONEY
...,...,...,...,...,...
95,96,Waspada Masalah yang Timbul akibat Telat Ganti...,"JAKARTA, KOMPAS.com - Oli mesin pada mobil den...",06/09/2024,OTOMOTIF
96,97,"Sosok Faisal Basri di Mata Para Tokoh, Ekonom ...","JAKARTA, KOMPAS.com - Ekonom senior Faisal Bas...",06/09/2024,MONEY
97,98,"Pendaftaran CPNS Diperpanjang 4 Hari, Pelamar ...","JAKARTA, KOMPAS.com - Pemerintah telah memperp...",06/09/2024,MONEY
98,99,"Harga Emas Terbaru Pegadaian, Jumat 6 Septembe...","JAKARTA, KOMPAS.com - Pegadaian menyediakan be...",06/09/2024,MONEY


**Praproses Teks**
fungsi

In [ ]:
# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

# Menghapus tanda baca, angka, dan simbol
def clean_punct(text):
    clean_spcl = re.compile('[/(){}\|@,;_]')
    clean_symbol = re.compile('[^0-9a-z]')
    clean_number = re.compile('[0-9]')
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    text = clean_number.sub('', text)
    return text

# Menghaps double atau lebih whitespace
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

# Menghapus stopwords
def clean_stopwords(text):
    stopword = set(stopwords.words('indonesian'))
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

# Stemming with Sastrawi
def sastrawistemmer(text):
    factory = StemmerFactory()
    st = factory.create_stemmer()
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

function clean_lower digunakan untuk merubah semua kata atau huruf menjadi huruf kecil semua

function clean_punct digunakan untuk menghapus karakter, simbol, dan angka

function _normalize_whitespace digunakan untuk menghapus spasi yang double atau lebih dari 2 spasi

function clean_stopwords digunakan untuk menghilangkan kata yang tidak perlu (kata hubung, kata tambahan dll)

function sastrawistemmer digunakan untuk proses stemming (mendapatkan kata dasar dari suatu kata)

**Clean Lower**

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses case folding
main_df['lwr'] = main_df['Isi Berita'].apply(clean_lower)
casefolding=pd.DataFrame(main_df['lwr'])
casefolding


,lwr
0,"jakarta, kompas.com - surat izin mengemudi (s..."
1,"jakarta, kompas.com - banyak pembaca yang ingi..."
2,"jakarta, kompas.com - penting bagi calon nasab..."
3,kompas.com - pt kereta api indonesia (kai) div...
4,"jakarta, kompas.com - otoritas jasa keuangan (..."
...,...
95,"jakarta, kompas.com - oli mesin pada mobil den..."
96,"jakarta, kompas.com - ekonom senior faisal bas..."
97,"jakarta, kompas.com - pemerintah telah memperp..."
98,"jakarta, kompas.com - pegadaian menyediakan be..."


**Clean Punct**

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan tanda baca
main_df['clean_punct'] = main_df['lwr'].apply(clean_punct)
main_df['clean_punct']

,clean_punct
0,jakarta kompas com surat izin mengemudi sim...
1,jakarta kompas com banyak pembaca yang ingin...
2,jakarta kompas com penting bagi calon nasaba...
3,kompas com pt kereta api indonesia kai divre...
4,jakarta kompas com otoritas jasa keuangan oj...
...,...
95,jakarta kompas com oli mesin pada mobil deng...
96,jakarta kompas com ekonom senior faisal basr...
97,jakarta kompas com pemerintah telah memperpa...
98,jakarta kompas com pegadaian menyediakan ber...



**Normalize Whitespace**

In [ ]:
main_df['clean_double_ws'] = main_df['clean_punct'].apply(_normalize_whitespace)
main_df['clean_double_ws']


,clean_double_ws
0,jakarta kompas com surat izin mengemudi sim wa...
1,jakarta kompas com banyak pembaca yang ingin t...
2,jakarta kompas com penting bagi calon nasabah ...
3,kompas com pt kereta api indonesia kai divre i...
4,jakarta kompas com otoritas jasa keuangan ojk ...
...,...
95,jakarta kompas com oli mesin pada mobil dengan...
96,jakarta kompas com ekonom senior faisal basri ...
97,jakarta kompas com pemerintah telah memperpanj...
98,jakarta kompas com pegadaian menyediakan berba...



**Clean Stopwords**

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan stopwords
main_df['clean_sw'] = main_df['clean_double_ws'].apply(clean_stopwords)
main_df['clean_sw']

,clean_sw
0,jakarta kompas com surat izin mengemudi sim wa...
1,jakarta kompas com pembaca diskon motor honda ...
2,jakarta kompas com calon nasabah saldo minimal...
3,kompas com pt kereta api indonesia kai divre i...
4,jakarta kompas com otoritas jasa keuangan ojk ...
...,...
95,jakarta kompas com oli mesin mobil transmisi c...
96,jakarta kompas com ekonom senior faisal basri ...
97,jakarta kompas com pemerintah memperpanjang pe...
98,jakarta kompas com pegadaian menyediakan jenis...


**Stemming dengan Sastrawi**

In [ ]:
# Buat kolom tambahan untuk data description yang telah dilemmatization
main_df['desc_clean_stem'] = main_df['clean_sw'].apply(sastrawistemmer)
main_df['desc_clean_stem']

100%|██████████| 103/103 [00:02<00:00, 42.59it/s]


,desc_clean_stem
0,jakarta kompas com surat izin kemudi sim wajib...
1,jakarta kompas com baca diskon motor honda sep...
2,jakarta kompas com calon nasabah saldo minimal...
3,kompas com pt kereta api indonesia kai divre i...
4,jakarta kompas com otoritas jasa uang ojk blok...
...,...
95,jakarta kompas com oli mesin mobil transmisi c...
96,jakarta kompas com ekonom senior faisal basri ...
97,jakarta kompas com perintah panjang daftar sel...
98,jakarta kompas com gadai sedia jenis emas bata...


**Vector Space Model**

**Fitting kata untuk mendapatkan Term Frequency pada semua Dokumen**

In [ ]:
count_vectorizer = CountVectorizer()
corpus = main_df['desc_clean_stem'].tolist()
x_count = count_vectorizer.fit_transform(corpus)
print("Shape:", x_count.shape)

Shape: (100, 3555)


**Perhitungan Term Frequency dan Invers Document Frequency**

In [ ]:
vocabulary = count_vectorizer.get_feature_names_out().tolist()
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                 ('tfid', TfidfTransformer())]).fit(corpus)

tf = pipe['count'].transform(corpus).toarray()
tfidf = []
for row in tf:
  data = {}
  for j, col in enumerate(row):
    data[vocabulary[j]] = col * pipe['tfid'].idf_[j]
  tfidf.append(data)

**DataFrame**

In [ ]:
cat_df = main_df.loc[:, "Kategori Berita"]
tfidf_df = pd.DataFrame(tfidf)

tfidf_df = pd.concat([cat_df, tfidf_df], axis=1)
tfidf_df

,Kategori Berita,aaion,aali,abadi,abai,abenkh,abnormal,absurd,ac,acapkali,...,za,zad,zag,zaman,zarco,zenix,zero,zig,zigzag,zona
0,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**menyimpan ke CSV**

In [ ]:
tfidf_df.to_csv('data_berita_vsm.csv', index=False)

In [ ]:
cd/content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
ls

'Crawling data dari website berita online.aurellia zhullvita_22-200.ipynb'
'crawling data di kompas_22-200_aurellia zhullvita'
 NLPAPP-220411100200_catatan
'Salinan dari instalasi3.ipynb'
'Salinan dari instalasi.ipynb'
'Salinan dari UAS_KK_Aurellia Zhullvita_22-200.ipynb'
'scrab_berita kompas'
 tugas_2_22_200_aurellia_zhullvita.ipynb
 tugas2_22-200_aurellia_zhullvita.ipynb
'tugas crawling data_detikcom.ipynb'
'UAS_KK_Aurellia Zhullvita_22-200.ipynb'
 Untitled0.ipynb


In [ ]:
!jupyter nbconvert --to html tugas2_22-200_aurellia_zhullvita.ipynb

[NbConvertApp] Converting notebook tugas2_22-200_aurellia_zhullvita.ipynb to html
[NbConvertApp] Writing 671867 bytes to tugas2_22-200_aurellia_zhullvita.html
